In [5]:
from aiohttp import ClientError
import boto3
from botocore.exceptions import EndpointConnectionError
from pymongo import MongoClient, errors
import time

# Netflix dataset structure (sample record)
record = {
    'UserID': '101',
    'Name': 'John Doe',
    'SubscriptionType': 'Premium',
    'MonthlyRevenue': 29.99,
    'JoinDate': '2024-01-01',
    'LastPaymentDate': '2024-11-30',
    'Country': 'USA',
    'Age': 30,
    'Gender': 'Male',
    'Device': 'Laptop',
    'PlanDuration': '1 year'
}

# DynamoDB Configuration
dynamodb = boto3.client(
    'dynamodb',
    endpoint_url="http://localhost:8000",  # Local DynamoDB
    region_name='us-east-1',
    aws_access_key_id='fakeMyKeyId',  # Fake credentials
    aws_secret_access_key='fakeSecretAccessKey'
)

# MongoDB setup
client = MongoClient('mongodb://localhost:27017')  
db = client['MongoDB']  
collection = 'NetflixUserbase'


def check_dynamodb_availability():
    print("\nChecking DynamoDB availability...")
    try:
        # Write operation
        sample_record = {
            'UserID': {'S': '999'},
            'SubscriptionType': {'S': 'Basic'},
            'MonthlyRevenue': {'S': '9.99'},
            'JoinDate': {'S': '2024-12-01'},
            'LastPaymentDate': {'S': '2024-12-01'},
            'Country': {'S': 'USA'},
            'Age': {'N': '25'},
            'Gender': {'S': 'Female'},
            'Device': {'S': 'Mobile'},
            'PlanDuration': {'S': '1 month'}
        }

        # Write Operation
        start_write = time.time()
        dynamodb.put_item(TableName='NetflixUserbase', Item=sample_record)
        write_latency = time.time() - start_write

        # Read Operation
        start_read = time.time()
        response = dynamodb.get_item(
            TableName='NetflixUserbase',
            Key={'UserID': {'S': '999'}}
        )
        read_latency = time.time() - start_read

        # Verify Availability
        if 'Item' in response:
            print(f"DynamoDB is available.")
            print(f"Write Latency: {write_latency:.4f} seconds.")
            print(f"Read Latency: {read_latency:.4f} seconds.")
        else:
            print("DynamoDB is unavailable: Item not found.")

    except EndpointConnectionError as e:
        print(f"DynamoDB is unavailable: {e}")
    except ClientError as e:
        print(f"DynamoDB encountered a client error: {e}")


# Function to check MongoDB availability
def check_mongodb_availability():
    print("\nChecking MongoDB availability...")
    try:
        # Write operation
        start_write = time.time()
        collection.insert_one(record)
        write_latency = time.time() - start_write

        # Read operation
        start_read = time.time()
        result = collection.find_one({'UserID': record['UserID']})
        read_latency = time.time() - start_read

        if result:
            print(f"MongoDB is available.")
            print(f"Write Latency: {write_latency:.4f} seconds.")
            print(f"Read Latency: {read_latency:.4f} seconds.")
        else:
            print("MongoDB is unavailable: Item not found.")
    except errors.ServerSelectionTimeoutError as e:
        print(f"MongoDB is unavailable: {e}")
    except Exception as e:
        print(f"MongoDB encountered an error: {e}")

# Main Execution
if __name__ == "__main__":
    print("Starting availability checks...\n")
    check_dynamodb_availability()
    check_mongodb_availability()
    print("\nAvailability checks completed.")


Starting availability checks...


Checking DynamoDB availability...
DynamoDB is available.
Write Latency: 0.0354 seconds.
Read Latency: 0.0064 seconds.

Checking MongoDB availability...
MongoDB encountered an error: 'str' object has no attribute 'insert_one'

Availability checks completed.
